## Introduction

This notebook presents a benchmark between Power Grid Model and the Tensor Power Flow. N.B., the performance gain of TPF is made purely out of the data formulation, no advantage of Intel MKL or CUDA is taken.

In [1]:
import warnings
from time import time
from tqdm import tqdm

import numpy as np
import power_grid_model as pgm
from generate_fictional_dataset import generate_fictional_grid_pgm_tpf
from plotter import BenchmarkPlotter
from power_grid_model.validation import errors_to_string, validate_batch_data, validate_input_data
from tensorpowerflow import GridTensor

warnings.filterwarnings("ignore")

Configurations for grids: PGM method dictionary for exact methods


In [2]:
pgm_method_dict = {
    "iterative_current": "PGM Iterative Current",
    "newton_raphson": "PGM Newton-Raphson",
}

Benchmark experiment function: 

In [3]:
def experiment(n_feeder=20, n_node_per_feeder=50, n_step=1000, log=False):
    def log_print(*args):
        if log:
            print(*args)

    # fictional grid parameters
    cable_length_km_min = 0.8
    cable_length_km_max = 1.2
    load_p_w_min = 0.4e6 * 0.8
    load_p_w_max = 0.4e6 * 1.2
    pf = 0.95

    load_scaling_min = 0.5
    load_scaling_max = 1.5

    # gen grid data
    fictional_dataset = generate_fictional_grid_pgm_tpf(
        n_node_per_feeder=n_node_per_feeder,
        n_feeder=n_feeder,
        cable_length_km_min=cable_length_km_min,
        cable_length_km_max=cable_length_km_max,
        load_p_w_max=load_p_w_max,
        load_p_w_min=load_p_w_min,
        pf=pf,
        n_step=n_step,
        load_scaling_min=load_scaling_min,
        load_scaling_max=load_scaling_max,
    )
    # unpack data
    pgm_dataset = fictional_dataset["pgm_dataset"]
    pgm_update_dataset = fictional_dataset["pgm_update_dataset"]
    tpf_node_data = fictional_dataset["tpf_grid_nodes"]
    tpf_line_data = fictional_dataset["tpf_grid_lines"]
    tpf_time_series_p = fictional_dataset["tpf_time_series_p"]
    tpf_time_series_q = fictional_dataset["tpf_time_series_q"]

    # validate data
    log_print(errors_to_string(validate_input_data(pgm_dataset)))
    log_print(errors_to_string(validate_batch_data(pgm_dataset, pgm_update_dataset)))

    res_pgm = []
    # create grids, run pf's and time them
    # pgm - all 4 methods
    pgm_methods = ["iterative_current", "newton_raphson"]
    for method in pgm_methods:
        pgm_start_time = time()
        model_instance = pgm.PowerGridModel(pgm_dataset)
        start = time()
        _ = model_instance.calculate_power_flow(
            symmetric=True,
            calculation_method=method,
            update_data=pgm_update_dataset,
            output_component_types=["node", "line"],
            max_iterations=10000,
        )
        end = time()
        pgm_end_time = time()
        res_pgm.append(end - start)
        log_print(f"{pgm_method_dict[method]}: {end - start}")
        log_print(f"Total time{pgm_method_dict[method]}: {pgm_end_time - pgm_start_time}")

    # tpf
    tpf_time_start = time()
    tpf_instance = GridTensor(
        node_file_path="",
        lines_file_path="",
        from_file=False,
        nodes_frame=tpf_node_data,
        lines_frame=tpf_line_data,
        gpu_mode=False,
    )
    tpf_time_end = time()

    tpf_functions = ["run_pf_tensor"]
    res_tpf = []
    for function_name in tpf_functions:
        start = time()
        _ = getattr(tpf_instance, function_name)(active_power=tpf_time_series_p, reactive_power=tpf_time_series_q)
        end = time()
        res_tpf.append(end - start)
        log_print(f"TensorPowerFlow.{function_name}: {end - start}")

    log_print(f"TensorPowerFlow instancing: {tpf_time_end - tpf_time_start}")

    return {"entry pgm": pgm_methods, "result pgm": res_pgm, "entry tpf": tpf_functions, "result tpf": res_tpf}

Now run the different experiment configurations.

In [ ]:
exp_options = [[20, 1], [20, 5], [20, 10], [20, 25], [40, 25], [40, 50]]
for n in tqdm([10, 100, 1000, 10000], desc="Overall Progress"):
    plotter = BenchmarkPlotter(n_steps=n)
    for option in tqdm(exp_options, desc=f"Processing n={n}"):
        res = experiment(option[0], option[1], n)
        plotter.add(res, option[0] * option[1])
    plotter.plot()